In [1]:
import numpy as np
import pymatgen
from pymatgen.ext.matproj import MPRester
import pymatgen.analysis.magnetism.analyzer as pg
import torch

In [2]:
mp = MPRester()

In [3]:
ids = []
for line in open('training_results/ids.txt'):
    ids.append(line.split()[0])

In [4]:
unique_mpids = list(set(ids))
len(ids), len(unique_mpids)

(92882, 23440)

In [5]:
test_mpids = np.load('mpid.npy')
test_vecs = np.load('vector2.npy')

In [6]:
len(test_mpids)

11393

In [9]:
len(set(test_mpids))

7142

In [6]:
# len(set(unique_mpids + list(test_mpids)))

In [27]:
def magnetization_per_mag_atoms(structure):
    mag_atoms = 0
    m = pg.CollinearMagneticStructureAnalyzer(structure)
    mag_atoms = 0
    for s,mag in zip(struct.species, m.magmoms):
        if s in m.types_of_magnetic_species:
            mag_atoms += 1
    if mag_atoms == 0:
        return 0.0
    return np.abs(m.total_magmoms) / mag_atoms

def mag_label(structure):
    m = pg.CollinearMagneticStructureAnalyzer(structure)
    return m.ordering.name

def mag_mag(structure):
    m = pg.CollinearMagneticStructureAnalyzer(structure)
    return m.magmoms

In [ ]:
# structures = []
mag_per_mag_atom = []

for i,mat_id in enumerate(test_mpids):
    if i % 100 == 0:
        print(i)
    try:
        struct = mp.query({'material_id': mat_id}, ['structure'])[0]['structure']
#         structures.append(struct)
        mag_per_mag_atom.append(magnetization_per_mag_atoms(struct))
    except:
        mag_per_mag_atom.append(None)
#         structures.append(None)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200


In [ ]:
mag_per_mag_atom = np.array(mag_per_mag_atom)
np.save(mag_per_mag_atom, "mag_per_mag_atom.npy")

In [28]:
labels = [mag_label(s) for s in structures]
magmoms = [mag_mag(s) for s in structures]

In [40]:
vec_orderings = ['NM', 'AFM', 'FM/FiM']
get_order = lambda x: vec_orderings[x]

In [43]:
pred_labels = list(get_order(x) for x in torch.argmax(torch.tensor(test_vecs), dim=1))

In [46]:
for i,l in enumerate(zip(pred_labels[:100], labels)):
    print(i, l)

0 ('NM', 'FM')
1 ('NM', 'NM')
2 ('NM', 'NM')
3 ('FM/FiM', 'FiM')
4 ('NM', 'NM')
5 ('NM', 'NM')
6 ('AFM', 'AFM')
7 ('NM', 'NM')
8 ('AFM', 'AFM')
9 ('FM/FiM', 'FM')
10 ('NM', 'NM')
11 ('FM/FiM', 'FM')
12 ('NM', 'NM')
13 ('FM/FiM', 'FiM')
14 ('FM/FiM', 'FM')
15 ('NM', 'NM')
16 ('FM/FiM', 'FM')
17 ('NM', 'NM')
18 ('AFM', 'FM')
19 ('FM/FiM', 'AFM')
20 ('AFM', 'FM')
21 ('NM', 'NM')
22 ('FM/FiM', 'AFM')
23 ('AFM', 'AFM')
24 ('FM/FiM', 'FiM')
25 ('FM/FiM', 'FiM')
26 ('FM/FiM', 'FM')
27 ('FM/FiM', 'AFM')
28 ('NM', 'NM')
29 ('AFM', 'AFM')
30 ('AFM', 'FiM')
31 ('AFM', 'AFM')
32 ('NM', 'NM')
33 ('FM/FiM', 'FiM')
34 ('NM', 'NM')
35 ('AFM', 'AFM')
36 ('FM/FiM', 'AFM')
37 ('FM/FiM', 'FiM')
38 ('AFM', 'AFM')
39 ('NM', 'NM')
40 ('NM', 'NM')
41 ('AFM', 'FM')
42 ('AFM', 'FM')
43 ('NM', 'NM')
44 ('NM', 'NM')
45 ('AFM', 'FiM')
46 ('NM', 'NM')
47 ('AFM', 'AFM')
48 ('NM', 'NM')
49 ('AFM', 'AFM')
50 ('NM', 'NM')
51 ('NM', 'NM')
52 ('AFM', 'AFM')
53 ('AFM', 'AFM')
54 ('NM', 'NM')
55 ('AFM', 'AFM')
56 ('FM/FiM'

In [84]:
i = 98 # 0, 18, 19, 20, 22, 30, 36, 41, 42, 45, 56, 61, 69, 74, 75, 76, 77, 78, 80, 83, 89, 96, 98
mag_per_mag_atom[i], magmoms[i], pred_labels[i], labels[i], test_vecs[i]

(0.00024999999999999935,
 array([-2.76, -2.76,  2.76,  2.76,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ,  0.  ]),
 'FM/FiM',
 'AFM',
 array([0.16458626, 0.38976157, 0.44565217]))

In [30]:
for i in zip(test_mpids[0:100], mag_per_mag_atom, labels, magmoms):
    print(i)

('mp-1209180', 0.001, 'FM', array([0.   , 0.   , 0.001, 0.001, 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   ]))
('mp-1213150', 0.0, 'NM', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0]))
('mp-1143651', 0.0, 'NM', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0]))
('mp-759616', 3.1315294117647063, 'FiM', array([ 4.232,  4.559,  4.237,  4.157,  4.657,  4.609,  4.591,  4.693,
        4.218,  4.575,  4.541,  4.184, -0.129, -0.149, -0.193, -0.157,
       -0.185,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   ,  0.   ]))
('mp-697734', 0.0, 'NM', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [15]:
s = mp.query({'material_id': test_mpids[1]}, ['structure'])[0]['structure']

In [16]:
m = pg.CollinearMagneticStructureAnalyzer(s)

In [19]:
m.types_of_magnetic_species, m.magmoms

((),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0]))

In [18]:
s

Structure Summary
Lattice
    abc : 7.7945145102347455 7.7945145102347455 7.630805980762779
 angles : 63.097716390695744 63.097716390695744 89.8446276725775
 volume : 356.5980038107378
      A : 5.504076 5.519022 0.0
      B : -5.504076 5.519022 0.0
      C : 0.0 4.876266 5.869517
PeriodicSite: Cs (-3.3249, 9.1762, 4.4021) [0.1980, 0.8020, 0.7500]
PeriodicSite: Cs (3.3249, 6.7381, 1.4674) [0.8020, 0.1980, 0.2500]
PeriodicSite: Nd (3.0342, 9.1762, 4.4021) [0.7756, 0.2244, 0.7500]
PeriodicSite: Nd (-3.0342, 6.7381, 1.4674) [0.2244, 0.7756, 0.2500]
PeriodicSite: W (0.0647, 11.4246, 4.5851) [0.6958, 0.6840, 0.7812]
PeriodicSite: W (-0.0647, 4.4898, 1.2844) [0.3042, 0.3160, 0.2188]
PeriodicSite: W (0.0647, 6.9279, 4.2191) [0.3160, 0.3042, 0.7188]
PeriodicSite: W (-0.0647, 8.9864, 1.6504) [0.6840, 0.6958, 0.2812]
PeriodicSite: O (-0.6861, 10.2952, 3.3133) [0.6210, 0.7456, 0.5645]
PeriodicSite: O (0.6861, 5.6191, 2.5562) [0.3790, 0.2544, 0.4355]
PeriodicSite: O (-0.6861, 8.0573, 5.4910) [0.25